### Este es un cuaderno simple para construir y visualizar árboles de decisión.

Acompaña al Capítulo 2 del libro.

Autora: Viviana Acquaviva, con contribuciones de Jake Postiglione y Olga Privman. Traducido por Manuel Pichardo Marcano y Genaro Suárez.
Algunos créditos de visualización é inspiración:

https://towardsdatascience.com/scikit-learn-decision-trees-explained-803f3812290d

https://medium.com/@rnbrown/creating-and-visualizing-decision-trees-with-python-f8e8fa394176


In [ ]:
import numpy as np #importar el paquete de numpy 

import matplotlib #importar el paquete de matplotlib 

import matplotlib.pyplot as plt #importar el paquete de matplotlib donde puedes crear graficas 

import matplotlib.patches as mpatches

import pandas as pd #new! #importar el paquete pandas para poder abrir el archivo

from sklearn.model_selection import train_test_split #no la usamos aquí, ¡pero es una función útil!

from sklearn.tree import DecisionTreeClassifier #Así es como se importan métodos

from sklearn import metrics #esto nos dará acceso a las métricas de evaluación

In [ ]:
font = {'size'   : 20}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
#Aquí hay un montón de paquetes solo para fines de visualización: se pueden omitir si son problemáticos

from io import StringIO
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz #puedes usar esto si las otras líneas te dan problemas

### Usaremos una selección de información de http://phl.upr.edu/projects/habitable-exoplanets-catalog/data/database

### Comenzamos leyendo el conjunto de datos usando pandas.

In [ ]:
LearningSet = pd.read_csv('../data/HPLearningSet.csv') #conjunto de aprendizaje

In [ ]:
!head '../data/HPLearningSet.csv'

In [ ]:
LearningSet

In [ ]:
LearningSet = LearningSet.drop(LearningSet.columns[0], axis=1) #queremos soltar la primera columna del conjunto de aprendizaje

La estructura que creamos se llama un marco de datos.

Es agradable porque podemos referirnos a las columnas con sus nombres y también con sus índices, y se ve ordenado.

In [ ]:
LearningSet

In [ ]:
LearningSet[['P_NAME','S_MASS']] #manera conveniente de acceder a las columnas

In [ ]:
LearningSet.P_NAME #aquí viene otro 

### Escojamos los mismos conjuntos de entrenamiento y prueba que teníamos en el ejercicio.

Tenga en cuenta el uso de ".iloc" (ubicación de números enteros) para acceder a índices en marcos de datos.

In [ ]:
TrainSet =  LearningSet.iloc[:13,:]  #normalmente esto sucedería al azar, usando la función train_test_split

TestSet = LearningSet.iloc[13:,:]

In [ ]:
TrainSet

In [ ]:
TestSet

### Dividimos los conjuntos de entrenamiento y prueba  en las características y etiquetas.

In [ ]:
Xtrain = TrainSet.drop(['P_NAME','P_HABITABLE'],axis=1) #características para el conjunto de entrenamiento

Xtest = TestSet.drop(['P_NAME','P_HABITABLE'],axis=1) #características para el conjunto de prueba

In [ ]:
ytrain = TrainSet.P_HABITABLE #objetivo para el conjunto de entrenamiento 

ytest = TestSet.P_HABITABLE  #objetivo para el conjunto de prueba

### ¡Y estamos listos para ajustar el modelo con nuestro árbol de decisiones!

Nota: Las características siempre se permutan aleatoriamente en cada división. Por tanto, el split mejor encontrado puede variar, incluso con los mismos datos de entrenamiento, si la mejora del criterio es idéntica para varios splits enumerados durante la búsqueda del mejor split.

Para obtener un comportamiento determinista durante el ajuste, se debe corregir random_state.


In [ ]:
model = DecisionTreeClassifier(random_state = 3) #Así es como especificamos qué método nos gustaría usar y cualquier parámetro.

model.fit(Xtrain, ytrain) #Esta pequeña línea es cómo construimos modelos en sklearn.

### Finalmente, podemos visualizar el árbol.

In [ ]:
dot_data = StringIO()
export_graphviz(
            model,
            out_file =  dot_data,
            feature_names = ['Stellar Mass (M*)', 'Orbital Period (d)', 'Distance (AU)'],
            class_names = ['Not Habitable','Habitable'],
            filled = True,
            rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
nodes = graph.get_node_list()

for node in nodes:
    if node.get_label():
        values = [int(ii) for ii in node.get_label().split('value = [')[1].split(']')[0].split(',')]
        values = [255 * v / sum(values) for v in values]
        
        values = [int(255 * v / sum(values)) for v in values]
            
        if values[0] > values[1]:
            alpha = int(values[0] - values[1])
            alpha = '{:02x}'.format(alpha) #se convierte a hexadecimal
            color = '#20 B2 AA'+str(alpha)
        else:
            alpha = int(values[1] - values[0])
            alpha = '{:02x}'.format(alpha)
            color = '#FF 00 FF'+str(alpha)
        node.set_fillcolor(color)

graph.set_dpi('300')

Image(graph.create_png())

#Imagen(gráfico.write_png('Gráfico.png'))

### Esta es una visualización alternativa, que solo se basa en el paquete sklearn.

In [ ]:
from sklearn import tree

plt.figure(figsize=(40,20))  #personalizar de acuerdo al tamaño de tu árbol

tree.plot_tree(model, feature_names = ['Stellar Mass (M*)', 'Orbital Period (d)', 'Distance (AU)'], class_names = ['Not Habitable','Habitable'])

plt.show()

### También podemos visualizar las divisiones y luego responder algunas preguntas.




In [ ]:
plt.figure(figsize=(12,8))

cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#20B2AA','#FF00FF'])

#Ahora trazará el conjunto de trenes y los puntos de ajuste de prueba
#Ahora trazáramos los puntos de los conjuntos de entrenamiento y de prueba

plt.scatter(TrainSet['S_MASS'], TrainSet['P_PERIOD'], marker = '*',\
            c = TrainSet['P_HABITABLE'], s = 100, cmap=cmap, label = 'Train')

plt.scatter(TestSet['S_MASS'], TestSet['P_PERIOD'], marker = 'o',\
            c = TestSet['P_HABITABLE'], s = 100, cmap=cmap, label = 'Test')

plt.yscale('log')

plt.xlabel('Mass of Parent Star (Solar Mass Units)')

plt.ylabel('Period of Orbit (days)');

#Se puede agregar las divisiones en la grafica

plt.axvline(x=0.83, linewidth =1, ls = '-', label = '1st split', c='k')

plt.axhline(y=4.891, xmin = 0, xmax = 0.655, linewidth =1, ls = '--', label = '2nd split',c='k')

plt.text(0.845, 10**3, '1st split', fontsize=14)
         
plt.text(0.65, 6, '2nd split', fontsize=14)

#Agregar leyenda, incluyendo los objetos sin etiqueta

bluepatch = mpatches.Patch(color='#20B2AA', label='Not Habitable')

magentapatch = mpatches.Patch(color='#FF00FF', label='Habitable')

plt.legend();

ax = plt.gca()

predhab = mpatches.Rectangle((0,4.891),0.83,ax.get_ylim()[1], 
                        fill = True,
                        color = '#FF00FF',
                        alpha = 0.3)

prednothab1 = mpatches.Rectangle((0.83,ax.get_ylim()[0]),ax.get_xlim()[1],ax.get_ylim()[1], 
                        fill = True,
                        color = '#20B2AA',
                        alpha = 0.3)

prednothab2 = mpatches.Rectangle((0,ax.get_ylim()[0]),0.83,4.891-ax.get_ylim()[0], 
                        fill = True,
                        color = '#20B2AA',
                        alpha = 0.3)

leg = ax.get_legend()
leg.legendHandles[2].set_color('k')
leg.legendHandles[3].set_color('k')

plt.gca().add_patch(predhab)
plt.gca().add_patch(prednothab1)
plt.gca().add_patch(prednothab2)

leg = ax.get_legend()
leg.legendHandles[2].set_color('k')
leg.legendHandles[3].set_color('k')


plt.legend(handles=[leg.legendHandles[2],leg.legendHandles[3], magentapatch, bluepatch],\
           loc = 'upper left', fontsize = 14);


### Registro de aprendizaje
    
P: ¿Cuál es la precisión (porcentaje de clasificaciones correctas) en el conjunto de entrenamiento?

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
100%
</p>
</details>

<br/>

P: ¿Qué tal en el conjunto de prueba? (Tendrá que ejecutar el ejemplo de prueba a través del árbol o mirar la figura de arriba.)

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
3/5 or 60%   
</p>
</details>

In [ ]:
#Queremos, por supuesto, poder responder las preguntas en código también.

ypred = model.predict(Xtest) #cómo generar etiquetas predichas en el conjunto de prueba

In [ ]:
metrics.accuracy_score(ytest, ypred) #resultado del conjunto de prueba

In [ ]:
metrics.accuracy_score(ytrain, model.predict(Xtrain)) #resultado del conjunto de entrenamiento

### Nuestra reflexión final será un ejercicio para elegir un conjunto de entrenamiento y prueba diferente. 


In [ ]:
TrainSet2 = LearningSet.iloc[5:,:] #elegimos los primeros 5 objetos para prueba, 5:18 para entrenamiento

TestSet2 = LearningSet.iloc[:5,:]

### Volver a realizar el proceso de nuevo...

In [ ]:
Xtrain2 = TrainSet2.drop(['P_NAME','P_HABITABLE'],axis=1)

Xtest2 = TestSet2.drop(['P_NAME','P_HABITABLE'],axis=1)

ytrain2 = TrainSet2.P_HABITABLE

ytest2 = TestSet2.P_HABITABLE

### ¡Y estamos listos para ajustar el modelo nuevamente con nuestro árbol de decisiones!

model = DecisionTreeClassifier(random_state=3)

model.fit(Xtrain2,ytrain2)

### Ahora podemos visualizar el nuevo árbol:

In [ ]:
dot_data = StringIO()
export_graphviz(
            model,
            out_file =  dot_data,
            feature_names = ['Stellar Mass (M*)', 'Orbital Period (d)', 'Distance (AU)'],
            class_names = ['Not Habitable','Habitable'],
            filled = True,
rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
nodes = graph.get_node_list()

for node in nodes:
    if node.get_label():
        values = [int(ii) for ii in node.get_label().split('value = [')[1].split(']')[0].split(',')]
        values = [255 * v / sum(values) for v in values]
        
        values = [int(255 * v / sum(values)) for v in values]
            
        if values[0] > values[1]:
            alpha = int(values[0] - values[1])
            alpha = '{:02x}'.format(alpha) #se convierte a hexadecimal
            color = '#20 B2 AA'+str(alpha)
        else:
            alpha = int(values[1] - values[0])
            alpha = '{:02x}'.format(alpha)
            color = '#FF 00 FF'+str(alpha)
        node.set_fillcolor(color)

graph.set_dpi('300')

Image(graph.create_png())

#Imagen(gráfico.write_png('Gráfico.png'))

### Como se puede ver, ¡esto es bastante diferente a lo que teníamos antes!

### Revisión de aprendizaje

P: ¿Cuál es la precisión (porcentaje de clasificaciones correctas) en el conjunto de entrenamiento?

<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>
100%
</p>
</details>

<br/>

P: ¿Qué tal en el conjunto de prueba? <i>(¡Comprueba tu código en la celda de abajo!)</i>

In [ ]:
# Ingrese el código en esta celda



<details>
<summary style="display: list-item;">¡Haga clic aquí para obtener la respuesta!</summary>
<p>

```python
metrics.accuracy_score(ytest2, model.predict(Xtest2))
```
 
<i>Resultado</i>
    
1.0

</p>
</details>

### Saquemos juntos algunas conclusiones…

- ¿Fortalezas del algoritmo DT? ¡Fácil, rápido e interpretable!

- ¿Limitaciones? Solo puede dividir una característica a la vez; requiere ingeniería de funciones si desea considerar combinaciones de funciones.

- Posibles preocupaciones? Este conjunto de datos es probablemente demasiado pequeño para sacar conclusiones; el hecho de que las notas de prueba fluctúen tanto en respuesta a las diferentes divisiones del conjunto de entrenamiento y prueba es una indicación de esto.